In [1]:
%%time

import glob

import geopandas as gpd
import matplotlib.pyplot as plt
import numba
import numpy as np
import pandas as pd
from libpysal.graph import read_parquet
from sklearn.preprocessing import PowerTransformer, RobustScaler, StandardScaler
from sklearn.neighbors import KDTree
from core.cluster_validation import print_distance, generate_neigbhourhood_groups
from core.utils import char_names
import momepy as mm

CPU times: user 3.12 s, sys: 92.6 ms, total: 3.21 s
Wall time: 932 ms


In [2]:
region_id = 'barcelona'
buildings_dir = streets_dir = enclosures_dir = tessellations_dir = graph_dir = '../data/barcelona/'
chars_dir = '../data/barcelona/chars/'
old_file = '../data/barcelona/old_barcelona.parquet'

## Download old data


In [3]:
import requests 
def download_url(url, save_path, chunk_size=128):
    r = requests.get(url, stream=True)
    with open(save_path, 'wb') as fd:
        for chunk in r.iter_content(chunk_size=chunk_size):
            fd.write(chunk)

# download_url('https://figshare.com/ndownloader/files/38855088', '../data/barcelona/old_barcelona.parquet')

# download_url('https://opendata-ajuntament.barcelona.cat/data/dataset/6fd03b36-7503-42f0-9082-9040cb812423/resource/a11a534d-4aeb-497f-b6bd-2e170b4724aa/download', '../data/barcelona/streets_barcelona.zip')

# download_url(
#     "https://opendata-ajuntament.barcelona.cat/data/dataset/808daafa-d9ce-48c0-925a-fa5afdb1ed41/resource/cd800462-f326-429f-a67a-c69b7fc4c50a/download", '../data/barcelona/admin_units.zip')

In [ ]:
import osmnx as ox

In [ ]:
streets = gpd.read_file('../data/barcelona/streets_barcelona.zip', layer = 'BCN_GrafVial_Trams_ETRS89_SHP')

In [ ]:
bounds = streets.to_crs(4326).total_bounds
tags = {'natural': ['water', 'coastline', 'spring']}

water = ox.geometries_from_bbox(bounds[3], bounds[1], bounds[2], bounds[0], tags)
water = water.to_crs(streets.crs)
water[['natural', 'geometry']].to_parquet("../data/barcelona/water.pq")

tags = {'railway': True}
railway = ox.geometries_from_bbox(bounds[3], bounds[1], bounds[2], bounds[0], tags)
railway = railway.to_crs(streets.crs)
railway = railway[railway.geom_type == 'LineString']
railway = railway[railway.tunnel != 'yes']
railway = railway[~railway.railway.isin(['miniature', 'tram'])]
railway[['railway', 'geometry']].to_parquet("../data/barcelona/railway.pq")

## Process old data through the current pipeline

In [5]:
streets = gpd.read_file('../data/barcelona/streets_barcelona.zip', layer = 'BCN_GrafVial_Trams_ETRS89_SHP')

In [6]:
water = gpd.read_parquet('../data/barcelona/water.pq')
railway = gpd.read_parquet('../data/barcelona/railway.pq')
admin = gpd.read_file("../data/barcelona/0301100100_UNITATS_ADM_POLIGONS.json")

In [7]:
extended_railway = mm.extend_lines(railway, 30, target=streets, extension=.1)

In [8]:
enclosures = mm.enclosures(streets, limit=admin.iloc[[0]], additional_barriers=[extended_railway])

In [9]:
old_gdf = gpd.read_parquet(old_file)

In [10]:
buildings = old_gdf[['uID', 'buildings']]

In [11]:
tessellation = old_gdf[['tessellation', 'tID', 'buildings']]

In [13]:
inp, res = enclosures.sindex.query(tessellation.representative_point().geometry, predicate='within')

In [14]:
inp == np.arange(82375)

array([ True,  True,  True, ...,  True,  True,  True])

In [15]:
tessellation['enclosure_index'] = res

/home/krasen/miniconda3/envs/urban_taxonomy/lib/python3.12/site-packages/geopandas/geodataframe.py:1528: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [16]:
buildings.columns = ['uID', 'geometry']
buildings = buildings.set_geometry('geometry')
buildings.dropna().to_crs(epsg=3035).to_parquet(buildings_dir + f"buildings_{region_id}.parquet")

In [17]:
new_index = np.array(tessellation.index.values)
new_index[tessellation.isna()['buildings']] = -new_index[tessellation.isna()['buildings']]
tessellation = tessellation[['tessellation', 'tID', 'enclosure_index']]
tessellation.columns = ['geometry', 'tID', 'enclosure_index']
tessellation = tessellation.set_geometry('geometry')
tessellation.index = new_index

tessellation.to_crs(epsg=3035).to_parquet(tessellations_dir + f"tessellation_{region_id}.parquet")

In [18]:
streets.to_crs(epsg=3035)[['geometry']].to_parquet(streets_dir + f"streets_{region_id}.parquet")

In [19]:
enclosures.to_crs(epsg=3035).to_parquet(enclosures_dir + f"enclosure_{region_id}.parquet")

### Push data through pipeline


In [3]:
from core.generate_ngraphs import process_region_graphs

In [4]:
%%time
process_region_graphs(
    region_id,
    graph_dir,
    buildings_dir,
    streets_dir,
    enclosures_dir,
    tessellations_dir,
)

Built tess graph knn=1
Built buildings graph knn=1
Built streets graph knn=1
Built enclosure graph knn=1
Built nodes graph knn=1
CPU times: user 20.4 s, sys: 568 ms, total: 21 s
Wall time: 20.9 s


/home/krasen/libpysal/libpysal/weights/weights.py:1685: UserWarning: The weights matrix is not fully connected: 
 There are 41 disconnected components.
  w = W(neighbors, weights, ids, silence_warnings=silence_warnings)


## Characters

In [5]:
from core.generate_chars import (process_single_region_chars, process_building_chars,
                                 process_tessellation_chars, process_street_chars)

In [6]:
# process_tessellation_chars(
#     region_id,
#     graph_dir,
#     buildings_dir,
#     streets_dir,
#     enclosures_dir,
#     tessellations_dir,
#     chars_dir
# )

In [7]:
%%time
process_single_region_chars(
    region_id,
    graph_dir,
    buildings_dir,
    streets_dir,
    enclosures_dir,
    tessellations_dir,
    chars_dir
)

2024-08-09 15:52:56.546973 ----Processing ------ barcelona
Processing streets


/home/krasen/miniconda3/envs/urban_taxonomy/lib/python3.12/site-packages/shapely/measurement.py:72: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)
/home/krasen/miniconda3/envs/urban_taxonomy/lib/python3.12/site-packages/shapely/measurement.py:72: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)


Processing enclosures
Processing buildings


/home/krasen/momepy/momepy/functional/_shape.py:890: RuntimeWarning: invalid value encountered in arccos
  angles = np.arccos(cosine_angle)
/home/krasen/momepy/momepy/functional/_shape.py:890: RuntimeWarning: invalid value encountered in arccos
  angles = np.arccos(cosine_angle)
/home/krasen/momepy/momepy/functional/_shape.py:576: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(90 - degrees[true_angles]))
/home/krasen/momepy/momepy/functional/_shape.py:890: RuntimeWarning: invalid value encountered in arccos
  angles = np.arccos(cosine_angle)
/home/krasen/momepy/momepy/functional/_shape.py:737: RuntimeWarning: Mean of empty slice
  return Series({"mean": np.nanmean(dists), "std": np.nanstd(dists)})
/home/krasen/miniconda3/envs/urban_taxonomy/lib/python3.12/site-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


Processing tessellation


/home/krasen/miniconda3/envs/urban_taxonomy/lib/python3.12/site-packages/shapely/measurement.py:72: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)
/home/krasen/miniconda3/envs/urban_taxonomy/lib/python3.12/site-packages/shapely/measurement.py:72: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)


CPU times: user 6min 37s, sys: 1.38 s, total: 6min 39s
Wall time: 6min 38s


In [8]:
tessellation = gpd.read_parquet(chars_dir + f"tessellations_chars_{region_id}.parquet")
buildings = gpd.read_parquet(chars_dir + f"buildings_chars_{region_id}.parquet")
enclosures = gpd.read_parquet(chars_dir + f"enclosures_chars_{region_id}.parquet")
streets = gpd.read_parquet(chars_dir + f"streets_chars_{region_id}.parquet")
nodes = gpd.read_parquet(chars_dir + f"nodes_chars_{region_id}.parquet")


merged = pd.merge(
    tessellation.drop(columns=["geometry"]),
    buildings.drop(columns=["nodeID", "geometry", 'nID']),
    right_index=True,
    left_index=True,
    how="left",
)

merged = merged.merge(
    enclosures.drop(columns="geometry"),
    right_index=True,
    left_on="enclosure_index",
    how="left",
)

merged = merged.merge(streets.drop(columns="geometry"), on="nID", how="left")
merged = merged.merge(nodes.drop(columns="geometry"), on="nodeID", how="left")

merged = merged.drop(
    columns=[
        "nID",
        "eID",
        "nodeID",
        "mm_len",
        "cdsbool",
        "node_start",
        "node_end",
        "x",
        "y",
        "enclosure_index",
        # "id",
        # "osm_id",
        # "index",  ## maybe keep
    ]
)
merged = merged.set_index(tessellation.index)

In [9]:
from core.utils import used_keys

In [10]:
primary = merged[list(used_keys.keys())]
primary.shape

(82375, 63)

In [11]:
primary.to_parquet(chars_dir + f'primary_chars_{region_id}.parquet')

## Comparison

In [12]:
import lonboard
from core.utils import char_names
from mapclassify import classify
import matplotlib as mpl
from lonboard.colormap import apply_continuous_cmap
from sidecar import Sidecar
import momepy as mm
from core.utils import used_keys

In [13]:
old_prague_data = pd.read_parquet(old_file)

In [14]:
new_prague_data = pd.read_parquet(chars_dir + f'primary_chars_{region_id}.parquet')

In [15]:
graph_keys = {
    
    "degree": "mtdDeg",
    "meshedness": "lcdMes",
    "local_closeness": "lcnClo",
    "proportion_3": "linP3W",
    "proportion_4": "linP4W",
    "proportion_0": "linPDE",
    "ldeAre": "ldkAre",
    "ldePer": "ldkPer",
    "lseCCo":"lskCCo",
    "lseERI":"lskERI",
    "lseCWA":"lskCWA",
    "lteOri": "ltkOri",
    "lteWNB": "ltkWNB",
    "lieWCe": "likWCe",
    "ltcWRE": "ltcWRB",
    "lieWBB": "likWBB", 
    "misRea": "ldsRea",
    "mdsAre": "ldsAre"
}

for c, new_c in graph_keys.items():

    if c in old_prague_data.columns:
        old_prague_data[new_c] = old_prague_data[c].copy()

In [16]:
vals = {}
for c in used_keys:

    old_c = c
    
    if old_c not in old_prague_data.columns:
        print(c, 'not in old data')
        continue
        
    new_values = new_prague_data[c].values
    old_values = old_prague_data[old_c].values

    # skip nas
    nas = np.isnan(new_values) | np.isnan(old_values)
    new_values = new_values[~nas]
    old_values = old_values[~nas]

    vals[c] = np.corrcoef(new_values, old_values)[0,1]

ltcBuA not in old data
midRea not in old data
midAre not in old data
likWBB not in old data


In [17]:
vals = pd.Series(vals)
non_correlated = vals[ vals < .80]
non_correlated.sort_values()

mdcAre    0.125303
ldsMSL    0.420367
ltcWRB    0.451673
ltbIBD    0.559449
sdsAre    0.563558
mtbSWR    0.597466
lddNDe    0.641135
ldsAre    0.733176
stbSAl    0.771552
ldsRea    0.773734
linWID    0.774684
dtype: float64

In [18]:
[used_keys[c] for c in non_correlated.sort_values().index]

['area covered by neighbouring cells',
 'mean segment length within 3 steps',
 'number of unique enclosures in ETC neigbhourhood',
 'mean inter-building distance',
 'area covered by edge-attached ETCs',
 'shared walls ratio of buildings',
 'local node density of street network',
 'reached total ETC area by local street network',
 'street alignment of building',
 'reached ETCs by local street network',
 'local degree weighted node density of street network']

In [17]:
c = 'mdcAre' # this is due to the tesellation topology issues
c = 'ldsMSL' # this is due to a bug in the momepy.SegmentsLengths code, if you pick .series it returns the sum
c = 'ltcWRB' # this is due to the tesellation topology issues
c = 'ltbIBD' # this is due to the tesellation topology issues
c = 'sdsAre' # because it uses all the edges that touch a tessellation in general is higher - network ratio - get nearest street 
c = 'mtbSWR' # we capped it at 1, old can be higher + we use a larger buffer
c = 'lddNDe' #  nodes and edges are pretty much the same...
c = 'sddAre' # no idea
...

Ellipsis

In [20]:
tessellation = gpd.read_parquet(chars_dir + f"tessellations_chars_{region_id}.parquet")
buildings = gpd.read_parquet(chars_dir + f"buildings_chars_{region_id}.parquet")
enclosures = gpd.read_parquet(chars_dir + f"enclosures_chars_{region_id}.parquet")
streets = gpd.read_parquet(chars_dir + f"streets_chars_{region_id}.parquet")

In [21]:
c = 'lddNDe' 


new_values = new_prague_data[c].values
old_values = old_prague_data[c].values

# skip nas
nas = np.isnan(new_values) | np.isnan(old_values)
new_values = new_values[~nas]
old_values = old_values[~nas]
print(np.corrcoef(new_values, old_values)[0,1], new_values.shape)

0.641135074747926 (82375,)


In [19]:
(tessellation['nID'].values == old_prague_data['edgeID_primary'].values).sum() / old_prague_data['edgeID_primary'].shape[0]

0.9699666160849773

In [20]:
(tessellation['nodeID'].values + 1 == old_prague_data['nodeID'].values).sum() / old_prague_data['nodeID'].shape[0]

0.9826039453717754

In [22]:
np.sum(old_values - new_values)

99.86145862208346

In [25]:
graph = mm.gdf_to_nx(streets, preserve_index=True)
graph = mm.node_degree(graph)
res = mm.node_density(graph, radius=5, verbose=False)

In [26]:
nodes, edges, sw = mm.nx_to_gdf(res, spatial_weights=True)

/home/krasen/libpysal/libpysal/weights/weights.py:1685: UserWarning: The weights matrix is not fully connected: 
 There are 41 disconnected components.
  w = W(neighbors, weights, ids, silence_warnings=silence_warnings)


In [27]:
nodes.index = nodes.index + 1

In [28]:
c = 'degree'

r =  old_prague_data[['nodeID', c]].groupby('nodeID').first()

In [29]:
assert (nodes.loc[r.index, c] == r.values[:, 0]).all()

In [30]:
nodes_w5 = mm.sw_high(k=5, weights=sw)

/tmp/ipykernel_30141/422387120.py:1: FutureWarning: `momepy.sw_high` is deprecated. Replace it with .higher_order() method of libpysal.graph.Graph or pin momepy version <1.0. This class will be removed in 1.0. 
  nodes_w5 = mm.sw_high(k=5, weights=sw)


In [31]:
res = mm.NodeDensity(nodes.reset_index(drop=True), edges, nodes_w5, verbose=False).series

/tmp/ipykernel_30141/2870891164.py:1: FutureWarning: Class based API like `momepy.NodeDensity` is deprecated. Replace it with `momepy.node_density` to use functional API instead or pin momepy version <1.0. Class-based API will be removed in 1.0. 
  res = mm.NodeDensity(nodes.reset_index(drop=True), edges, nodes_w5, verbose=False).series


In [32]:
assert np.allclose(res.values, nodes['node_density'].values)

In [124]:
plotting = streets.copy()

In [125]:
%%time
layer = lonboard.PathLayer.from_geopandas(plotting, width_min_pixels=1)

CPU times: user 58 ms, sys: 7.98 ms, total: 65.9 ms
Wall time: 65.9 ms


/home/krasen/miniconda3/envs/urban_taxonomy/lib/python3.12/site-packages/lonboard/_geoarrow/ops/reproject.py:97: UserWarning: Input being reprojected to EPSG:4326 CRS
  warnings.warn("Input being reprojected to EPSG:4326 CRS")


In [126]:
sc = Sidecar(title='Clusters')
m = lonboard.Map(layer, basemap_style=lonboard.basemap.CartoBasemap.Positron)
with sc:
    display(m)

In [134]:
column = c
classifier = classify(streets[column], 'equalinterval', k=20)
normalizer = mpl.colors.Normalize(0, classifier.bins.shape[0])
vals = normalizer(classifier.yb)
layer.get_color = apply_continuous_cmap(vals, mpl.colormaps['viridis'])

ValueError: arange: cannot compute length

In [73]:
barca_chars = ["stbCeA",
    "sdcLAL",
    "sdcAre",
    "sscCCo",
    "sscERI",
    "sicCAR",
    "mtbAli",
    "mtbNDi",
    "mtcWNe",
    "mdcAre",
    "ltbIBD",
    "stcSAl",
    "ltcWRE",
    "sdbAre",
    "sdbPer",
    "sdbCoA",
    "ssbCCo",
    "ssbCor",
    "ssbSqu",
    "ssbERI",
    "ssbElo",
    "ssbCCM",
    "ssbCCD",
    "mtbSWR",
    "ldbPWL",
    "stbSAl",
    "libNCo",
    "sdsLen",
    "sdsSPW",
    "sdsSPO",
    "sdsSWD",
    "sssLin",
    "sdsAre",
    "sisBpM",
    "misRea",
    "mdsAre",
    "ldsMSL",
    "degree",
    "meshedness",
    "proportion_3",
    "proportion_4",
    "proportion_0",
    "local_closeness",
    "ldsCDL",
    "xcnSCl",
    "mtdMDi",
    "lddNDe",
    "linWID",
    "sddAre",
    "ldeAre",
    "ldePer",
    "lseCCo",
    "lseERI",
    "lseCWA",
    "lteOri",
    "lteWNB",
    "lieWCe",]

In [77]:
new_b_chars = []
for c in barca_chars:

    if c in graph_keys:
        c = graph_keys[c]

    if c not in new_prague_data.keys():
        print(c)
        continue

    new_b_chars.append(c)

In [79]:
len(new_b_chars)

57